# 主題 05-2. 轉移學習的練習

讓我們回顧一下生命中第一個做出來的 CNN 圖形辨識模型...

## 1. 初始準備

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

C:\Users\oroch\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 2. 讀入 MNIST 數據庫

MNIST 是有一堆 0-9 的手寫數字圖庫。有 6 萬筆訓練資料, 1 萬筆測試資料。它是 "Modified" 版的 NIST 數據庫, 原來的版本有更多資料。這個 Modified 的版本是由 LeCun, Cortes, 及 Burges 等人做的。可以參考這個數據庫的[原始網頁](http://yann.lecun.com/exdb/mnist/)。

MNIST 可以說是 Deep Learning 最有名的範例, 它被 Deep Learning 大師 Hinton 稱為「機器學習的果蠅」。

### 2.1 由 Keras 讀入 MNIST
Keras 很貼心的幫我們準備好 MNIST 數據庫, 我們可以這樣讀進來 (第一周課程中已經讀過)。

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

我們可以看看資料的長相

In [5]:
print("There are %d training data with size %d x %d" %x_train.shape)
print("There are %d testing  data with size %d x %d" %x_test.shape)

There are 60000 training data with size 28 x 28
There are 10000 testing  data with size 28 x 28


### 2.3 輸入格式整理

我們現在要用 CNN 學手寫辨識。因為 CNN 模型的資料需要多一個 channel (通道數)，因此我們要用 `reshape` 調校一下。

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

為了後面需要，我們先將數字 0 和 1 的資料分別抓出來

In [7]:
x_train_01 = x_train[y_train <= 1]
x_test_01 = x_test[y_test <= 1]

並將 label 轉換成 one-hot encoding 的形式

In [8]:
y_train_10 = np_utils.to_categorical(y_train, 10)
y_test_10 = np_utils.to_categorical(y_test, 10)

y_train_01 = y_train[y_train <= 1]
y_train_01 = np_utils.to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test <= 1]
y_test_01 = np_utils.to_categorical(y_test_01, 2)

養成良好的習慣，適時的確認資料的大小以確保資料的一致性

In [9]:
x_train_01.shape, x_test_01.shape

((12665, 28, 28, 1), (2115, 28, 28, 1))

In [10]:
y_train_01.shape, y_test_01.shape

((12665, 2), (2115, 2))

# 3. 回顧 CNN 圖形辨識模型

經典的 CNN 圖形辨識模型 LeNet-5 是一個由兩層卷積層加三層全連接層所建立的神經網路，而在第二單元時，我們建立的 CNN 模型設定如下：

* 起始為 <span style="color:red;">3</span> 個 convolutional block
 + 每個 convolutional block 為 <span style="color:red;">1</span> 個 2D Convolution + ReLU + <span style="color:red;">1</span> 個 2D MaxPooling
 + 2D Convolution 的數量為 32, 64, 128
 + 每個 2D Convolution 的 `kernal_size` 為 3 或 (3, 3)，`padding` 使用 `same`
 + 每個 2D MaxPooling 的 `pool_size` 為 2 或 (2, 2)，`padding` 使用 `same`

* 將輸出結果 `Flatten` 後，接著兩層全連接層，神經元個數分別為 200 和 10 (<span style="color:red;">數字的類別總數)</span>

我們當時建立的，是一個具有三層卷積層加兩層全連接的神經網路，其實可以看成是 LeNet-5 的一種變形。

根據本單元的內容，我們可以使用下列方式使用 Sequential 重新建構第二單元的 CNN 模型。

In [11]:
# We put 3 conv. blocks together, called conv_layer.
conv_layer = [Conv2D(32, (3, 3), padding='same', input_shape=(28,28,1)),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2)),
              
              Conv2D(64, (3, 3), padding='same'),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2)),
              
              Conv2D(128, (3, 3), padding='same'),
              Activation('relu'),
              MaxPooling2D(pool_size=(2, 2))]

# We put Flatten, and 2 fully-connectd layers together, called fc_layer.
fc_layer = [Flatten(),
            Dense(200),
            Activation('relu'),
            Dense(10),
            Activation('softmax')]

model = Sequential(conv_layer + fc_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [12]:
model.load_weights('handwriting_weights_cnn.h5')

# 4. 保留前三層 convolutional layer 並進行轉移學習

在此，我們一樣將 MNIST 資料集將僅有 0, 1的部分取出來，我們希望透過轉移學習建立一個類似 LeNet-5 的 0, 1 圖形辨識模型。

請將下列三個 **None** 的部分進行修改，以透過轉移學習建立新的模型。

In [13]:
new_fc_layer = [Flatten(), 
                ### Design your own fully connected structures ###
                Dense(None),
                Activation(None),
                Dense(None),       ## Hint: how many classes in new dataset?
                ### Remember put correct number of unit for output ###
                Activation('softmax')]

model_0_to_1 = Sequential(conv_layer + None)
model_0_to_1.summary()

TypeError: can only concatenate list (not "NoneType") to list

請將下列的 **None** 進行修改，以將借過來的神經網路 **冷凍** 起來：

In [ ]:
for layer in None:
    layer.trainable = False

**冷凍**後的神經網路的 summary 會有些變化，你有發現嗎? ：)

In [ ]:
model_0_to_1.summary()

接著，我們來訓練這個(有一部分架構及權重跟別人借用的) 0, 1 手寫辨識模型吧！

In [ ]:
model_0_to_1.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

## 5. 訓練

In [ ]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size=100, epochs=5)

In [ ]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

In [ ]:
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

## 6. 恭喜你完成了第二個透過轉移學習得到的神經網路模型！

不難發現，如果模型大部分的權重已經訓練好並冷凍起來，則轉移學習可以大幅減少訓練時間且訓練會更快收斂，那麼，是否還有其他重要的模型建構技巧呢？

這個問題我們留待下個單元解答囉~ : )